### Pregunta 40: Para contenidos geolocalizados, ¿Cual es el contenido mas cercano del que fue editado mas recientemente? ¿Y la diferencia entre sus tiempos de edicion?

In [1]:
import pandas as pd
import numpy as np

In [2]:
columnas=['id','revision_timestamp']
contenidos_df=pd.read_csv("contents.csv",usecols=columnas,infer_datetime_format=True)
contenidos_df.dropna(inplace=True)
contenidos_df['revision_timestamp']=pd.to_datetime(contenidos_df['revision_timestamp'])

Se toma como punto de referencia temporal el momento en el cual se ejecuta el notebook

In [3]:
from datetime import datetime
ahora=datetime.now()

In [4]:
contenidos_df['revision_timestamp']=pd.to_datetime(contenidos_df.revision_timestamp).dt.tz_convert(None)
contenidos_df['revision_timestamp']=contenidos_df['revision_timestamp']-ahora
contenidos_df['dif_minutos']=contenidos_df['revision_timestamp'].astype('timedelta64[m]')
contenidos_df=contenidos_df.loc[:,['id','dif_minutos']]
contenidos_df['dif_minutos']=contenidos_df['dif_minutos']*(-1)

In [5]:
columnas=["gt_page_id","gt_lat","gt_lon"]
geo_tags_df=pd.read_csv("geo_tags.csv", usecols=columnas)
geo_tags_df=geo_tags_df.rename(columns={"gt_page_id":"id"})

Puede ocurrir que el contenido mas recientemente alterado no tenga datos de geolocalizacion. Se va a conseguir entonces el contenido mas recientemente alterado del que tengamos informacion

In [6]:
geo_tags_df=geo_tags_df.merge(contenidos_df, on='id')
geo_tags_df=geo_tags_df.sort_values("dif_minutos", ascending = True)

In [7]:
geo_tags_df.head(1)

,id,gt_lat,gt_lon,dif_minutos
535116,650097,42.599722,-6.725556,60785.0


El contenido mas recientemente cambiado tiene el id 650097 

In [10]:
lat_mas_reciente=42.599722
long_mas_reciente=-6.725556

In [11]:
geo_tags_df=geo_tags_df.loc[geo_tags_df['id'] != 650097, :]

In [12]:
geo_tags_df['distancia_al_reciente']=np.sqrt( (geo_tags_df['gt_lat']-lat_mas_reciente )**2  + (geo_tags_df['gt_lon'] - long_mas_reciente)**2 )

In [13]:
geo_tags_df=geo_tags_df.sort_values("distancia_al_reciente", ascending = True)

In [14]:
geo_tags_df.head(1)

,id,gt_lat,gt_lon,dif_minutos,distancia_al_reciente
231911,6102755,42.599722,-6.725556,262114.0,0.0


Entonces el contenido más cercano al más recientemente editado es el contenido 6102755, y tiene una diferencia de 247418 minutos (Aproximadamente 172 dias) de tiempo entre ambos.